Code used to estimate the force constant to use for harmonic potentials given the number of ions in simulation and the desired maximum concentration

## Imports

In [1]:
import numpy as np
import math
import sys
import mpmath
sys.modules['sympy.mpmath'] = mpmath
from openmm.unit import AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
from openmm.unit import kelvin, bar, litre, kilojoule_per_mole, mole, nanometer, angstrom, kilocalorie_per_mole,molar,molal
from openmm.unit import Quantity, Unit
import sympy as sp
from sympy import symbols, Function, log, Sum
from sympy.abc import x, y, z
import scipy.integrate
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.optimize import root


## Simulation Setup

In [2]:
R = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

In [3]:
#Volume in L of Box
w=4.8*nanometer
l=4.8*nanometer
h=14.4*nanometer
vol=(w*l*h)
volL=vol.in_units_of(litre)
print(f'Volume is {volL} L')

#Particles of water in box
waters=(55.55*molar)*volL*AVOGADRO_CONSTANT_NA
print(f'Number of water molecules in box: {math.ceil(waters)}')

Volume is 3.31776e-22 L L
Number of water molecules in box: 11099


In [4]:
def solute_particles(num_water_molecules, molality_molkg):
    """
    Calculate the number of solute molecules required to achieve a target molality.

    Parameters:
    - num_water_molecules (int): Total number of water molecules in the system.
    - molality_molkg (float): Target molality in mol/kg.

    Returns:
    - int: Number of solute molecules needed.
    """
    AVOGADRO = 6.02214076e23  # molecules/mol
    WATER_MOLAR_MASS = 18.01528  # g/mol
    KG_PER_GRAM = 1e-3  # Conversion factor

    # Calculate the mass of water in grams
    mass_water_grams = (num_water_molecules / AVOGADRO) * WATER_MOLAR_MASS

    # Convert mass of water to kilograms
    mass_water_kg = mass_water_grams * KG_PER_GRAM

    # Use molality definition: mol/kg = moles_solute / mass_water_kg
    moles_solute = molality_molkg * mass_water_kg

    # Convert moles of solute to number of molecules
    num_solute_molecules = int(round(moles_solute * AVOGADRO))

    return num_solute_molecules


In [5]:
molalities=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

In [6]:
solute_particle_counts = []
for i in molalities:
    particles_needed = solute_particles(num_water_molecules=math.ceil(waters), molality_molkg=i)
    solute_particle_counts.append(particles_needed)
    print(f"Number of solute molecules needed for {i} molal: {particles_needed}")

Number of solute molecules needed for 0.1 molal: 20
Number of solute molecules needed for 0.2 molal: 40
Number of solute molecules needed for 0.3 molal: 60
Number of solute molecules needed for 0.4 molal: 80
Number of solute molecules needed for 0.5 molal: 100
Number of solute molecules needed for 0.6 molal: 120
Number of solute molecules needed for 0.7 molal: 140
Number of solute molecules needed for 0.8 molal: 160
Number of solute molecules needed for 0.9 molal: 180
Number of solute molecules needed for 1.0 molal: 200


## Constants and variables to modify according to simulation setup

In [7]:
# System composition
Cmax_target = 0.5 * mole/litre # maximum molal concentration desired in units of molar
N_ions = 33 #number of ions corresponding to desired cmax

# Simulation box
L_z = 14.4 * nanometer # total length of box in z direction
Lx, Ly, Lz = np.array([4.8, 4.8, 14.4]) * nanometer

# Simulation temperature
T = 300 * kelvin

## Numerical Approximation

In [8]:
def calc_C_max_from_spring_const(k : Quantity) -> Quantity:
    A = Lx * Ly
    K = k / (2 * R * T)
    Ksq = np.sqrt(K._value) * (1*K.unit).sqrt()

    Cmax = (N_ions/A * Ksq) / (np.sqrt(np.pi) * erf(Lz / (2 * Ksq)))
    Cmax = (Cmax / AVOGADRO_CONSTANT_NA).in_units_of(molar)

    return Cmax

def Cmax_for_roots(k : float) -> float:
    return calc_C_max_from_spring_const(
        k * kilojoule_per_mole/nanometer**2
        ) - Cmax_target

In [9]:
#Estimate force constant
soln = root(lambda k : Cmax_for_roots(k)._value, np.float64(0.0))
print('Estimated force constant: ',soln.x,"\n")

Estimated force constant:  [0.69265056] 



/home/bamo6610/miniconda3/envs/calculus/lib/python3.10/site-packages/openmm/unit/quantity.py:414: RuntimeWarning: divide by zero encountered in reciprocal
  return self * pow(other, -1.0)


In [10]:
# Check force constant returns desired maximum concentration
Cmax = calc_C_max_from_spring_const(
    k=soln.x * kilojoule_per_mole/nanometer**2
    )
print('Cmax from spring constant: ',Cmax)

Cmax from spring constant:  [0.5] M


In [11]:
# Check force constant returns desired maximum concentration
Cmax_check = calc_C_max_from_spring_const(
    k=0.68 * kilojoule_per_mole/nanometer**2
    )
print('Cmax from spring constant: ',Cmax_check)

Cmax from spring constant:  0.4954129616936893 M


In [12]:
L_x=Lx.value_in_unit(nanometer)
L_y=Ly.value_in_unit(nanometer)
L_z=Lz.value_in_unit(nanometer)


In [17]:
R=8.31446261815324
T=300
k=0.68
N_s=34
ideal_norm = np.sqrt(np.pi*R*T/(2*1000*k)) #analytical solution
print(ideal_norm)
cmax_ideal=N_s/(L_x*L_y*ideal_norm)
print(cmax_ideal)
# Cmax_I = (cmax_ideal / AVOGADRO_CONSTANT_NA).in_units_of(molar)
# print(Cmax_I)

2.4003977004926202
0.6147708124122915
